In [17]:
#Dependencies
import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import MySQLdb
import mysql.connector

In [18]:
#Extract
housing_development = pd.read_csv('Assets/Data/Affordable_Rental_Housing_Developments.csv')
census_data = pd.read_csv('Assets/Data/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
vacant_building = pd.read_csv('Assets/Data/Vacant_and_Abandoned_Buildings_-_Violations.csv')

In [19]:
#clean/transform
housing_final = housing_development.dropna()
census_final = census_data.dropna()
vacant_final = vacant_building.dropna()

In [51]:
# vacant_building
census_data.head()
#housing_development.tail()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [21]:
census_data.at[77, 'Community Area Number']=0
#census_data

In [22]:
#Merging Census Data and housing development by Communit Area Number
Combined_data = pd.merge(housing_final, census_final, how='outer', on=['Community Area Number'])

In [23]:
# cleaning up ..
data = Combined_data.dropna()
data = data.drop(['COMMUNITY AREA NAME'], axis=1)

In [31]:
# connect to the database
database_path = "housing_data.db" #whatever you type in here will be the name of the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [32]:
#Double check it worked
inspector = inspect(engine)
inspector.get_table_names()

['housing_data']

In [39]:
#insert the table into the db
data.to_sql(name='housing_data', con=engine, if_exists='replace', index=False)

In [40]:
#Double check it worked
inspector = inspect(engine)
inspector.get_table_names()

['housing_data']

In [57]:
database_path = "housing_data.db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

Base = automap_base()
Base.prepare(engine, reflect=True)
Data = Base.classes.data()

AttributeError: data